### Data Preperation

In [72]:
#%% 1. Library setup
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import copy

In [73]:
DATANAME = "ITC_groundfloor.ply"
pcd = o3d.io.read_point_cloud("../Data/" + DATANAME)


In [74]:
# Print basic info
print(pcd)

# Print number of points
print("Number of points:", np.asarray(pcd.points).shape[0])

# Show the first few points
print("First 5 points:\n", np.asarray(pcd.points)[:5])

PointCloud with 334992 points.
Number of points: 334992
First 5 points:
 [[2.55192887e+05 4.73200869e+05 3.05235004e+01]
 [2.55192988e+05 4.73200812e+05 3.05227509e+01]
 [2.55193038e+05 4.73200783e+05 3.05232506e+01]
 [2.55192937e+05 4.73200840e+05 3.05249996e+01]
 [2.55192985e+05 4.73200872e+05 3.05240002e+01]]


In [75]:
# shift point cloud to bypasss large coordinates approxximation
pcd_center = pcd.get_center()
print("pcd_center:", pcd_center)
print("Type:", type(pcd_center))

pcd_center: [2.55203955e+05 4.73205983e+05 3.21217287e+01]
Type: <class 'numpy.ndarray'>


In [76]:
pcd.translate(-pcd_center)

PointCloud with 334992 points.

In [77]:
o3d.visualization.draw_geometries([pcd])

### Random Sampling
At the matrix level, the decimation acts by keeping points for every n-th row depending on the n factor. Of course, this is made based on how the points are stored in the file. Slicing a point cloud with open3d It is pretty straightforward. To shorten and parametrize the expression, you can write the lines:

In [78]:
retained_ratio = 0.2 # keep 20% of the points
sampled_pcd = pcd.random_down_sample(retained_ratio)

In [79]:
# Visualize the sampled point cloud
o3d.visualization.draw_geometries([sampled_pcd], window_name = "Random Sampling")

### Statistical outlier removal
Using an outlier filter on 3D point cloud data can help identify and remove any data points significantly different from the rest of the dataset. These outliers could result from measurement errors or other factors that can skew the analysis. By removing these outliers, we can get a more valid representation of the data and better adjust algorithms. However, we need to be careful not to delete valuable points.

In [80]:
nn = 16 # How many neightbors are considered to calculate the averate distance for a given point
std_multiplier = 10 # threshold based on standard deviation. the lower the number, the more aggressive the filter will be

filtered_pcd, filtered_idx = pcd.remove_statistical_outlier(nn, std_multiplier)

In [81]:
# colour outliers in red
outliers = pcd.select_by_index(filtered_idx, invert=True)
outliers.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([filtered_pcd, outliers])

### Point Cloud Voxel Sampling
The grid subsampling strategy is based on the division of the 3D space in regular cubic cells called voxels. For each cell of this grid, we only keep one representative point, and this point, the representative of the cell, can be chosen in different ways. When subsampling, we keep that cell's closest point to the barycenter.

In [82]:
voxel_size = 0.05
pcd_downsampled = filtered_pcd.voxel_down_sample(voxel_size = voxel_size)

In [83]:
o3d.visualization.draw_geometries([pcd_downsampled])

### Normals extraction
A point cloud normal refers to the direction of a surface at a specific point in a 3D point cloud. It can be used for segmentation by dividing the point cloud into regions with similar normals, for example. In our case, normals will help identify objects and surfaces within the point cloud, making it easier to visualize. And it is an excellent opportunity to introduce a way to compute such normals semi-automatically. We first define the average distance between each point in the point cloud and its neighbors:

In [84]:
nn_distance = 0.05

Then we use this information to extract a limited max_nn points within a radius radius_normals to compute a normal for each point in the 3D point cloud:



In [85]:
radius_normals=nn_distance*4
pcd_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn=16), fast_normal_computation=True)

In [86]:
pcd_downsampled.paint_uniform_color([0.6, 0.6, 0.6])
o3d.visualization.draw_geometries([pcd_downsampled,outliers])

### RANSAC
The RANSAC algorithm, short for RANdom SAmple Consensus, is a powerful tool for handling data that contains outliers, which is often the case when working with real-world sensors. The algorithm works by grouping data points into two categories: inliers and outliers. By identifying and ignoring the outliers, you can focus on working with reliable inliers, making your analysis more effective.

In [17]:
nn_distance = np.mean(pcd.compute_nearest_neighbor_distance())

In [18]:
distance_threshold = 0.1 # max distance a point can be from the plane model, and still be an inlier
ransac_n = 3 # number of points to sample for generating a plane model
num_iterations = 1000 # number of iterations

In [19]:
plane_model, inliers = pcd.segment_plane(distance_threshold=distance_threshold,ransac_n=3,num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

Plane equation: -0.00x + -0.00y + 1.00z + 1.35 = 0


In [20]:
inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)

#Paint the clouds
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])

#Visualize the inliers and outliers
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

### Scaling 3D: Multi-Order RANSAC
Our philosophy will be very simple. We will first run RANSAC multiple times (let say n times) to extract the different planar regions constituting the scene. Then we will deal with the “floating elements” through Euclidean Clustering (DBSCAN). It means that we have to make sure we have a way to store the results during iterations. 


In [21]:
# Empty dictionary to hold results of iterations
segment_models={} # Plane parameters
segments={} # Planar regions

In [22]:
max_plane_idx=10 # number of iterations

In [23]:
# seperate inliers from outliers. Store inliers in segments dictionary.
rest=pcd
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(
    distance_threshold=0.1,ransac_n=3,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print("pass",i,"/",max_plane_idx,"done.")

pass 0 / 10 done.
pass 1 / 10 done.
pass 2 / 10 done.
pass 3 / 10 done.
pass 4 / 10 done.
pass 5 / 10 done.
pass 6 / 10 done.
pass 7 / 10 done.
pass 8 / 10 done.
pass 9 / 10 done.


In [24]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])

### Euclidean Cluster DBSCAN
The DBSCAN algorithm involves scanning through each point in the dataset and constructing a set of reachable points based on density. This is achieved by analyzing the neighborhood of each point and including it in the region if it contains enough points. The process is repeated for each neighboring point until the cluster can no longer expand. Points that do not have enough neighbors are labeled as noise, making the algorithm robust to outliers. 

In [93]:
epsilon = 0.15
min_cluster_points = 5

In [94]:
rest=pcd
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(
    distance_threshold=0.1,ransac_n=3,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    labels = np.array(segments[i].cluster_dbscan(eps=epsilon, min_points=min_cluster_points)) # DBSCAN clustering
    candidates=[len(np.where(labels==j)[0]) for j in np.unique(labels)]
    best_candidate=int(np.unique(labels)[np.where(candidates== np.max(candidates))[0]])
    rest = rest.select_by_index(inliers, invert=True) + segments[i].select_by_index(list(np.where(labels!=best_candidate)[0]))
    segments[i]=segments[i].select_by_index(list(np.where(labels== best_candidate)[0]))
    segments[i].paint_uniform_color(list(colors[:3]))
    print("pass",i,"/",max_plane_idx,"done.")


C:\Users\nchan\AppData\Local\Temp\ipykernel_16748\1254871236.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  best_candidate=int(np.unique(labels)[np.where(candidates== np.max(candidates))[0]])


pass 0 / 10 done.
pass 1 / 10 done.
pass 2 / 10 done.
pass 3 / 10 done.
pass 4 / 10 done.
pass 5 / 10 done.
pass 6 / 10 done.
pass 7 / 10 done.
pass 8 / 10 done.
pass 9 / 10 done.


In [95]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])

### Euclidean Clusterign Refinement
Okay, time to evade the loop, and work on the remaining points assigned to therest variable, that are not yet attributed to any segment. Let us first get a visual grasp on what we are talking about:

In [90]:
o3d.visualization.draw_geometries([rest])

In [29]:
labels = np.array(pcd.cluster_dbscan(eps=0.1, min_points=10)) # -1 is noise. 0-n are the clusters

In [89]:
max_label = labels.max() 
colors = plt.get_cmap("tab20")(labels / (max_label 
if max_label > 0 else 1))
colors[labels < 0] = 0
rest.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([rest])

### Voxel Grid Generation


In [31]:
voxel_size=0.5

In [32]:
# Calculate the extents of the point cloud
min_bound = pcd.get_min_bound()
max_bound = pcd.get_max_bound()

In [33]:
# concate the segements from RANSAC
pcd_ransac=o3d.geometry.PointCloud()
for i in segments:
    pcd_ransac += segments[i]

In [34]:
# Extract voxel grid from the RANSAC segments
voxel_grid_structural = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd_ransac, voxel_size=voxel_size)

In [35]:
# same thing for remaining elements
rest.paint_uniform_color([0.1, 0.1, 0.8])
voxel_grid_clutter = o3d.geometry.VoxelGrid.create_from_point_cloud(rest, voxel_size=voxel_size)

In [88]:
o3d.visualization.draw_geometries([voxel_grid_clutter,voxel_grid_structural])

### Spacial Modeling
In indoor modeling applications, the voxel-based representation of point clouds plays a pivotal role in capturing and analyzing the geometric properties of complex environments. As the scale and complexity of point cloud datasets increase, it becomes essential to delve deeper into voxel segmentation techniques to extract meaningful structures and facilitate higher-level analysis. Let us thus define a function that fits a voxel grid and return both filled and empty spaces:

In [37]:
def fit_voxel_grid(point_cloud, voxel_size, min_b=False, max_b=False):
    # Determine the minimum and maximum coordinates of the point cloud
    if type(min_b) == bool or type(max_b) == bool:
        min_coords = np.min(point_cloud, axis=0)
        max_coords = np.max(point_cloud, axis=0)
    else:
        min_coords = min_b
        max_coords = max_b
        # Calculate the dimensions of the voxel grid
        grid_dims = np.ceil((max_coords - min_coords) / voxel_size).astype(int)
        # Create an empty voxel grid
        voxel_grid = np.zeros(grid_dims, dtype=bool)
        # Calculate the indices of the occupied voxels
        indices = ((point_cloud - min_coords) / voxel_size).astype(int)
        # Mark occupied voxels as True
        voxel_grid[indices[:, 0], indices[:, 1], indices[:, 2]] = True
    return voxel_grid, indices

In [40]:
min_bound = pcd.get_min_bound()
max_bound = pcd.get_max_bound()
grid_dims = np.ceil((max_bound - min_bound) / voxel_size).astype(int)
print("Grid dimensions:", grid_dims)

Grid dimensions: [60 47  8]


In [ ]:

voxel_size = 0.3

#get the bounds of the original point cloud
min_bound = pcd.get_min_bound()
max_bound = pcd.get_max_bound()

ransac_voxels, idx_ransac = fit_voxel_grid(pcd_ransac.points, voxel_size, min_bound, max_bound)
rest_voxels, idx_rest = fit_voxel_grid(rest.points, voxel_size, min_bound, max_bound)

#Gather the filled voxels from RANSAC Segmentation
filled_ransac = np.transpose(np.nonzero(ransac_voxels))

#Gather the filled remaining voxels (not belonging to any segments)
filled_rest = np.transpose(np.nonzero(rest_voxels))

#Compute and gather the remaining empty voxels
total = pcd_ransac + rest
total_voxels, idx_total = fit_voxel_grid(total.points, voxel_size, min_bound, max_bound)
empty_indices = np.transpose(np.nonzero(~total_voxels))


### Exporting 3D Datasets
 To visualize the results as shown below outside of Python with transparency, we need to export our data.

In [42]:
# export the segmented point cloud.
xyz_segments=[]
for idx in segments:
 print(idx,segments[idx])
 a = np.asarray(segments[idx].points)
 N = len(a)
 b = idx*np.ones((N,3+1))
 b[:,:-1] = a
 xyz_segments.append(b)

0 PointCloud with 105394 points.
1 PointCloud with 47753 points.
2 PointCloud with 28127 points.
3 PointCloud with 28063 points.
4 PointCloud with 11019 points.
5 PointCloud with 10944 points.
6 PointCloud with 18028 points.
7 PointCloud with 2988 points.
8 PointCloud with 4777 points.
9 PointCloud with 5763 points.


In [45]:
# save the remaining elements from DBSCAN
rest_labels = labels[-len(rest.points):]  # Use only the last N labels if rest is the last segment
rest_w_segments = np.hstack((np.asarray(rest.points), (rest_labels + max_plane_idx).reshape(-1, 1)))

In [46]:
xyz_segments.append(rest_w_segments)

In [48]:
np.savetxt("../Results/" + DATANAME.split(".")[0] + ".xyz", np.concatenate(xyz_segments), delimiter=";", fmt="%1.9f")

### Voxel Model Export

In [51]:
#Function created by F. Poux, License MIT.
#Please cite to reuse in your project

import numpy as np

def cube(c, s, compteur=0):
    """
    Generate vertices and faces for a cube.

    Parameters:
        c (numpy.ndarray): The center point of the cube.
        s (float): The side length of the cube.
        compteur (int, optional): Offset factor for the face indices. Default is 0.

    Returns:
        numpy.ndarray: An array containing vertex and face information.

    The function calculates the vertices and faces of a cube based on the center point (c),
    side length (s), and an optional compteur value for face offset. It constructs the
    vertices of the cube, defines the face vertices, assigns labels to the vertices and faces,
    and then combines them into an array representing the cube.

    Note: This function assumes that numpy has been imported as np.

    Example:
        center = np.array([0, 0, 0])
        side_length = 1.0
        cube_array = cube(center, side_length)
        print(cube_array)
    """
    # Calculate vertices of the cube
    v1 = c + s / 2 * np.array([-1, -1, 1])
    v2 = c + s / 2 * np.array([1, -1, 1])
    v3 = c + s / 2 * np.array([-1, 1, 1])
    v4 = c + s / 2 * np.array([1, 1, 1])
    v5 = c + s / 2 * np.array([-1, 1, -1])
    v6 = c + s / 2 * np.array([1, 1, -1])
    v7 = c + s / 2 * np.array([-1, -1, -1])
    v8 = c + s / 2 * np.array([1, -1, -1])

    # Define face vertices
    f1 = np.array([1, 2, 3])
    f2 = np.array([3, 2, 4])
    f3 = np.array([3, 4, 5])
    f4 = np.array([5, 4, 6])
    f5 = np.array([5, 6, 7])
    f6 = np.array([7, 6, 8])
    f7 = np.array([7, 8, 1])
    f8 = np.array([1, 8, 2])
    f9 = np.array([2, 8, 4])
    f10 = np.array([4, 8, 6])
    f11 = np.array([7, 1, 5])
    f12 = np.array([5, 1, 3])

    # Calculate vertex and face labels
    vcube = [v1, v2, v3, v4, v5, v6, v7, v8]
    ch = np.empty([8, 1], dtype=str)
    ch.fill('v')
    vertice = np.hstack((ch, vcube))

    faces = [f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12]
    faces = np.asarray(faces) + compteur * 8
    ch = np.empty([12, 1], dtype=str)
    ch.fill('f')
    faces = np.hstack((ch, faces))

    # Combine vertices and faces
    cube = np.append(vertice, faces, axis=0)
    return cube

In [52]:
def voxel_modelling(filename, indices, voxel_size):
    voxel_assembly=[]
    with open(filename, "a") as f:
        cpt = 0
        for idx in indices:
            voxel = cube(idx,voxel_size,cpt)
            f.write(f"o {idx}  \n")
            np.savetxt(f, voxel,fmt='%s')
            cpt += 1
            voxel_assembly.append(voxel)
    return voxel_assembly

In [53]:
vrsac = voxel_modelling("../RESULTS/ransac_vox.obj", filled_ransac, 1)
vrest = voxel_modelling("../RESULTS/rest_vox.obj", filled_rest, 1)
vempty = voxel_modelling("../RESULTS/empty_vox.obj", empty_indices, 1)